# Idiomatic Python

**Comprehensions**

  - concise list/dict/set construction with clear intent.

**Generators**

  - lazy pipelines; memory-light iteration.

**Unpacking**

  - readable assignment patterns and swaps; star captures.

**Context managers**

  - `with` for safe setup/teardown; custom managers.

**Decorators**

  - reusable wrappers (e.g., timing, caching) with `functools.wraps`.

**EAFP mindset**

  - "easier to ask forgiveness than permission" for clean, direct code.

## Comprehensions & Generator Expressions

In [1]:
data = [1, 2, 3, 4, 5]

In [2]:
squares = [x*x for x in data]

In [3]:
odds = {x: x*x for x in data if x % 2 ==1}

In [6]:
uniq_lengths = {len(str(x)) for x in data}

In [7]:
squares, odds, uniq_lengths

([1, 4, 9, 16, 25], {1: 1, 3: 9, 5: 25}, {1})

In [8]:
gen = (x*x for x in range(1_000_000))  # lazy: computes on demand

In [9]:
sum(next(gen) for _ in range(5))  # consume just a few                                  

30

## Unpacking and Assignment Patterns

In [14]:
trade = ("AAPL", 100, 190.5)

In [15]:
sym, qty, price = trade

In [16]:
sym, qty, price

('AAPL', 100, 190.5)

In [17]:
a, b = 10, 20; a, b = b, a; (a, b)

(20, 10)

In [18]:
head, *middle, tail = [1, 2, 3, 4, 5]

In [19]:
head, middle, tail

(1, [2, 3, 4], 5)

## Context Managers

In [20]:
from contextlib import contextmanager

In [21]:
@contextmanager
def timer(label: str):
    import time
    t0 = time.perf_counter()
    try:
        yield
    finally:
        dt = (time.perf_counter() - t0) * 1_000
        print(f"{label}: {dt:.2f} ms")  # timing info

In [22]:
with open("tmp.txt", "w") as f:
    _ = f.write("hello")

In [23]:
with open("tmp.txt") as f:
    f.read()

In [24]:
with timer("work"):
    _ = sum(i*i for i in range(50_000))

work: 13.88 ms


## Decorators (Intro)

In [25]:
import time

In [26]:
from functools import wraps, lru_cache

In [27]:
def timing(fn):
    @wraps(fn)
    def wrapper(*a, **k):
        t0 = time.perf_counter()
        out = fn(*a, **k)
        print(f"{fn.__name__} took {time.perf_counter()-t0:.6f}s")
        return out
    return wrapper

In [28]:
@timing
@lru_cache(maxsize=None)
def fib(n: int) -> int:
    return n if n < 2 else fib(n-1) + fib(n-2)

In [29]:
fib(20)

fib took 0.000006s
fib took 0.000007s
fib took 0.000236s
fib took 0.000004s
fib took 0.000287s
fib took 0.000004s
fib took 0.000332s
fib took 0.000004s
fib took 0.000377s
fib took 0.000004s
fib took 0.000420s
fib took 0.000004s
fib took 0.000470s
fib took 0.000004s
fib took 0.000514s
fib took 0.000005s
fib took 0.000775s
fib took 0.000004s
fib took 0.000830s
fib took 0.000004s
fib took 0.000874s
fib took 0.000004s
fib took 0.000918s
fib took 0.000004s
fib took 0.000962s
fib took 0.000004s
fib took 0.001008s
fib took 0.000004s
fib took 0.001059s
fib took 0.000004s
fib took 0.001103s
fib took 0.000004s
fib took 0.001150s
fib took 0.000005s
fib took 0.001263s
fib took 0.000004s
fib took 0.001307s
fib took 0.000004s
fib took 0.001352s


6765

In [30]:
fib(20)  # cached

fib took 0.000012s


6765

## EAFP: Easier to Ask Forgiveness than Permission

In [31]:
data = {"qty": "100", "price": "19.5"}

In [32]:
def trade_value(d):
    try:
        return int(d["qty"]) * float(d["price"])  # direct, readable
    except (KeyError, ValueError) as e:
        return None  # bad/missing fields

In [33]:
trade_value(data)

1950.0

## Mental Models & Pitfalls

**Readability first**
  - prefer a clear list/dict/set comprehension to a clever one; name helper functions when logic grows.

**Data flows, not steps**
  - think in terms of transforming collections, not indexing and counters.

**Lazy vs eager**
  - generators are streams; once consumed, they're empty. Wrap generator logic in a function to get a fresh stream each time.

**Bound now, not later**
  - closures in loops need the current value bound at definition (e.g., `lambda i=i:` ...).

**Ask forgiveness**
  - try the direct approach and catch specific failures; too many pre-checks obscure intent.

## Common Gotchas

**Generator exhaustion**
  - once consumed, a generator is empty; recreate it if you need to iterate again.

**Leaky comprehension vars (Py2 myth)**
  - in modern Python, the loop variable in a comprehension doesn't leak, but late binding still bites in closures inside loops.

**Overusing lambdas**
  - give complex logic a name; tiny helpers beat opaque inline functions.

**Shadowing names**
  - avoid reusing names like `list` or `sum`; keep builtins intact.

**Forgetting wraps**
  - missing `@wraps` breaks __name__ and docstrings; it also hurts debuggability.

## Exercises

**Filter and map in one**

From `xs = range(20)` build a list of squares of the even numbers using a single list comprehension.

In [34]:
xs = range(20)
even_squares = [x*x for x in xs if x % 2 == 0]
even_squares

[0, 4, 16, 36, 64, 100, 144, 196, 256, 324]

**Top-N unpack**

Given `vals = [5, 2, 9, 1, 7]`, sort descending and unpack the first three into `a, b, c`; colect the rest with a star.

In [37]:
vals = [5, 2, 9, 1, 7]
a, b, c, *rest = sorted(vals, reverse=True)
a, b, c, rest

(9, 7, 5, [2, 1])

**Timing context**

Write a `@contextmanager` called `timer(name)` that prints the elapsed milliseconds. Use it around a small computation.

In [43]:
from contextlib import contextmanager

@contextmanager
def timer(name: str):
    import time
    t0 = time.perf_counter()
    try:
        yield
    finally:
        print(f"{name}: ran in {time.perf_counter() - t0:.6f} s")

In [44]:
def factorial(n):
    import math
    if n < 2:
        return 1
    else:
        return math.prod(list(range(1,n+1)))

with timer("calc"):
    _ = factorial(12)

calc: ran in 0.000026 s


**Cached Fibonacci**

Add `@lru_cache(maxsize=None)` to a recursive Fibonacci and time the first vs second call.

In [46]:
from functools import wraps, lru_cache

def timing(fn):
    import time
    def wrapper(*a, **k):
        t0 = time.perf_counter()
        out = fn(*a, **k)
        print(f"{fn.__name__} took {time.perf_counter() - t0:.6f}s")
        return out
    return wrapper

In [47]:
@timing
@lru_cache(maxsize=None)
def fib(n: int) -> int:
    return n if n < 2 else fib(n-1) + fib(n-2)

In [48]:
fib(23)

fib took 0.000006s
fib took 0.000007s
fib took 0.000187s
fib took 0.000004s
fib took 0.000237s
fib took 0.000005s
fib took 0.000508s
fib took 0.000004s
fib took 0.000556s
fib took 0.000004s
fib took 0.000600s
fib took 0.000005s
fib took 0.000646s
fib took 0.000004s
fib took 0.000692s
fib took 0.000004s
fib took 0.000742s
fib took 0.000004s
fib took 0.000792s
fib took 0.000004s
fib took 0.000837s
fib took 0.000004s
fib took 0.000884s
fib took 0.000004s
fib took 0.000930s
fib took 0.000006s
fib took 0.001096s
fib took 0.000004s
fib took 0.001148s
fib took 0.000004s
fib took 0.001194s
fib took 0.000004s
fib took 0.001238s
fib took 0.000004s
fib took 0.001286s
fib took 0.000004s
fib took 0.001329s
fib took 0.000004s
fib took 0.001374s
fib took 0.000004s
fib took 0.001420s
fib took 0.000005s
fib took 0.001649s
fib took 0.000004s
fib took 0.001703s


28657

In [49]:
fib(23)

fib took 0.000012s


28657

**Safe lookup (EAFP)**

Implement `get_price(d)` that returns `float(d["price"])` or `None` for bad/missing data using EAFP.

In [53]:
def get_price(d):
    try:
        return float(d["price"])
    except (KeyError, ValueError) as e:
        return f"error getting price"

In [54]:
quote = {"sym": "AAPL", "price": "270.75"}
print(f"price: {get_price(quote)}")

price: 270.75


In [55]:
print(f"price: {get_price({})}")

price: error getting price


**Generator once**

Create a generator that yields the first 5 squares; show that iterating it twice gives results then empties. Fix by creating a function that returns a new generator each time.

In [58]:
gen = (x*x for x in range(1,6))

In [59]:
[next(gen) for _ in range(5)]

[1, 4, 9, 16, 25]

In [60]:
[next(gen) for _ in range(5)]

StopIteration: 

In [61]:
def sq_5():
    return (x*x for x in range(1,6))

In [63]:
[sq for sq in sq_5()]

[1, 4, 9, 16, 25]

In [64]:
[sq for sq in sq_5()]

[1, 4, 9, 16, 25]